In [1]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import torch
from torch import nn
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, Subset
from torchvision import transforms, datasets, models
from torchvision.models import resnet50, densenet121, inception_v3
from torchsummary import summary
from collections import OrderedDict
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import seaborn as sns
import wandb
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


In [2]:
class InferenceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.dataset = datasets.ImageFolder(root_dir)
        self.transform = transform
        self.classes = self.dataset.classes
        self.class_to_idx = self.dataset.class_to_idx
        
        # Count samples per class
        self.class_counts = [0] * len(self.classes)
        for _, label in self.dataset.samples:
            self.class_counts[label] += 1
        
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label
    
# Define your transformations
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

In [4]:
inference_dataset = InferenceDataset(root_dir='../data', transform=transform)
# Define your dataloader
inference_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False)

In [17]:
import torch
import torch.nn as nn
from torchvision.models import inception_v3

class InceptionBackbone(nn.Module):
	def __init__(self, path):
		super().__init__()
		base_model = inception_v3(weights=None, aux_logits=False)
		encoder_layers = list(base_model.children())
		self.backbone = nn.Sequential(*encoder_layers[:-1])
		
		state_dict = torch.load(path)
		backbone_state_dict = {k: v for k, v in state_dict.items() if k.startswith('backbone')}
		
		# Remove 'backbone.' prefix from keys
		backbone_state_dict = {k[len('backbone.'):]: v for k, v in backbone_state_dict.items()}
		
		self.backbone.load_state_dict(backbone_state_dict)
						
	def forward(self, x):
		x = self.backbone(x)
		return x

# Define your device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model architecture
model = InceptionBackbone(path='../PHASE#2/InceptionV3/best_checkpoints/best_InceptionV3.pt_epoch_30_fold_8.pth')

# Set the model to evaluation mode
model.eval()
model.to(device)

InceptionBackbone(
  (backbone): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (5): BasicConv2d(
      (conv): Conv2d(80, 192, kernel_size=(3, 3), 

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import inception_v3

class InceptionBackbone(nn.Module):
    def __init__(self, path):
        super().__init__()
        base_model = inception_v3(weights=None, aux_logits=False)
        encoder_layers = list(base_model.children())
        self.backbone = nn.Sequential(*encoder_layers[:-1])
        
        state_dict = torch.load(path)
        new_state_dict = {}
        for k, v in state_dict.items():
            new_state_dict[k[9:]] = v

        print(self.backbone.load_state_dict(new_state_dict))  # <All keys matched successfully>
                        
    def forward(self, x):
        x = self.backbone(x)
        return x

# Define your device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model architecture
model = InceptionBackbone(path='../PHASE#2/InceptionV3/best_checkpoints/best_InceptionV3.pt_epoch_30_fold_8.pth')

# Set the model to evaluation mode
model.eval()
model.to(device)

In [21]:
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F

# Inference
features = []
labels = []

for img, label in tqdm(inference_loader):
    img = img.to(device)
    with torch.no_grad():
        feature = model(img)
        features.append(feature.cpu().numpy())
        labels.append(label.cpu().numpy())

# Convert lists to numpy arrays
features = np.concatenate(features, axis=0)
labels = np.concatenate(labels, axis=0)

# Save features and labels to .npy files
np.save('features.npy', features)
np.save('labels.npy', labels)


100%|██████████| 8724/8724 [05:39<00:00, 25.67it/s]


In [22]:
features.shape, labels.shape

((8724, 2048, 1, 1), (8724,))